O presente notebook possui a intenção de prever o preço de imóveis do Rio de Janeiro. O algoritmo escolhido foi **KNeighborsRegressor**.

# Sobre o KNeighborsRegressor
Se trata de um algoritmo de regressão baseada nos k-vizinhos mais próximos. O alvo é previsto pela interpolação local dos alvos associados aos vizinhos mais próximos no conjunto de treinamento. Ele funciona da seguinte forma:

1. Selecionamos k vizinhos semelhantes mais próximos, usando algum cálculo de distância, com as quais queremos comparar.
1. Agora iremos calcular a semelhança entre cada vizinho e o nosso dado usando uma métrica de similaridade
1. Então classificamos cada vizinho usando nossa métrica de similaridade e selecionamos os primeiros k vizinhos.
1. Por fim, calculamos o valor médio dos k vizinhos semelhantes e o usamos como nosso valor de tabela.

# Depêndencias iniciais
Iremos agora importar as principais bibliotecas, sendo elas:

- [Numpy](https://numpy.org/): é um pacote fundamental para a computação científica com Python, usado principalmente para realizar cálculos em _arrays_ multidimensionais.
- [Pandas](https://pandas.pydata.org/): é um pacote que fornece estruturas de dados de alto desempenho e fáceis de usar, além de conter ferramentas de análise de dados.

In [0]:
import numpy as np
import pandas as pd

# O dataset
Nessa primeira parte, iremos carregar o _dataset_ na estrutura de dados que utilizaremos, que será o [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [0]:
# Lendo arquivo CSV
df_train = pd.read_csv("kn_train.csv")

In [0]:
# Conhecendo a quantidade de linhas e colunas do dataset, respectivamente
print("train:= ", df_train.shape)

In [0]:
df_train.columns

O _dataset_ possui as seguintes colunas (total de 26 colunas):

- **host_response_time**: tempo de resposta do host;
- **host_response_rate**: a taxa de resposta do host;
- **host_neighbourhood**: o bairro que o host se localiza;
- **host_listings_count**: número de outras listagens que o host possui;
- **neighbourhood**: o bairro no qual o espaço se localiza;
- **city**: a cidade onde fica o espaço;
- **state**: o estado em que o espaço fica;
- **zipcode**: o código postal em que fica o espaço;
- **latitude**: dimensão da latitude das coordenadas geográficas;
- **longitude**: dimensão de longitude das coordenadas geográficas;
- **property_type**: o tipo da propriedade;
- **room_type**: o tipo de espaço de convivência;
- **accommodates**: o número de pessoas que o aluguel pode acomodar;
- **bathrooms**: número de banheiros incluídos no aluguel;
- **bedrooms**: número de quartos incluídos no aluguel;
- **beds**: número de camas incluídas no aluguel;
- **bed_type**: o tipo da cama;
- **security_deposit**: depósito reembolsável, em caso de danos;
- **cleaning_fee**: taxa adicional usada para limpar o espaço após a saída do hóspede;
- **extra_people**: taxa adicional para pessoas extras; 
- **minimum_nights**: número mínimo de noites que um hóspede pode ficar no aluguel;
- **maximum_nights**: número máximo de noites que um hóspede pode ficar no aluguel;
- **number_of_reviews**: número de comentários que os hóspedes anteriores deixaram;
- **review_scores_rating**: pontuação do espaço;
- **reviews_per_month**: quantidade de reviews que o espaço recebe por mês;
- **price**: preço do aluguel.

## Explorando o dataset
Aqui iremos utilizar alguns métodos do pandas para conhecermos melhor o nosso _DataFrame_. Os métodos serão:
- **[head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html)**: mostra as primeiras _n_ linhas do nosso _dataset_. Por padrão serão as 5 primeiras linhas.
- **[info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html)**: imprime informações sobre o nosso _DataFrame_, incluindo o tipo de índice, os tipos de coluna, valores não nulos e uso de memória.
- **[describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)**: mostra estatísticas descritivas sobre as colunas do nosso _DataFrame_, como: a tendência central, a dispersão e a forma da distribuição de um conjunto de dados.

**Exercício**:
1. Utilize o método `head()` para verificar as primeiras 5 linhas do nosso _dataset_.
1. Utilize o método `info()` para ver informações sobre o nosso _dataset_.
1. Utilize o método `describe()` para verificar os valores estatísticos descritivos nosso _dataset_.

In [0]:
# Verificando as primeiras 5 linhas do nosso DataFrame
df_train._____()

In [0]:
# Mais informações sobre o DataFrame
df_train._____()

In [0]:
# Verificando detalhes estatísticos do DataFrame
df_train._____()

# Limpando, preparando e manipulando os dados
Nessa etapa, iremos realizar o processo de limpeza, preparação e manipulação dos dados do nosso _DataFrame_. Nesse primeiro momento temos que decidir quais as colunas que iremos usar no nosso modelo. Para isso iremos verificar as correlações das colunas com a coluna _price_ usando:
- **[corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)**: calcula a correlação entre todas as colunas. Usaremos o parâmetro **method** para especificar o método que iremos usar para calcular a correlação.
- **[sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sort_values.html)**: ordena os valores.

**Exercícios**:
1. Calcule a correlação entre todas as colunas usando o método `corr()`.
  1. Passe como valor para **method** a _string_ `'person'`.
1. Ordene os valores.

In [0]:
# Usamos a correlação de Pearson e ordenamos os valores
columns_corr = df_train._____(method=_____)['price']._____()
print(columns_corr)

## Filtrando as colunas que queremos
Após verificarmos a correlação das colunas entre si, iremos selecionar as 5 melhores colunas mais a coluna _price_.

Estamos usando:
- [**pandas.Series.index**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.index.html): pega os índices da `Series`.
- [**pandas.Series.tolist**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.tolist.html): converte para a estrutura de dados lista.

**Exercícios**:
1. Selecione as 5 melhores colunas.
1. Pegue os índices.
1. Converta a saída para `list`.

In [0]:
# Colunas do DataFrame que iremos usar
target_cols = columns_corr[_____]._____._____()
print(target_cols)

In [0]:
# Filtramos o DataFrame para apenas as colunas que queremos usar
clean_train = df_train[target_cols]
# Mostramos as dimensões do DataFrame
print(clean_train.shape)

In [0]:
# Algumas informações sobre o DataFrame
clean_train.info()

## Tratando os valores nulos
Anteriormente podemos notar que as colunas **beds**, **bedrooms** e **bathrooms** possuem valores nulos, portanto agora iremos adicionar a média dos valores da coluna em cada célula NA.

In [0]:
# Colunas que iremos preencher os valores NA com a média
fillna_mean = ['beds', 'bedrooms', 'bathrooms']

Para realizarmos a operação de adicionarmos a média em cada campo nulo, iremos utilizar duas funções:
- **[fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)**: preenche os valores de NA/NaN na coluna. Iremos especificar o valor que queremos aplicar nesses campos com o parâmetro **value**. O parâmetro **inplace** indica que não queremos retornar um novo _DataFrame_, que é para preencher o _DataFrame_ local.
- **[mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)**: retorna a média dos valores.

**Exercícios**:
1. Aplique o método `fillna()` na coluna dentro do `for`.
1. Como argumento para **value**, atribua a média daquela coluna usando o método `mean()`.
1. Passe `True` como argumento para **inplace**.

In [0]:
# Adicionamos as médias em cada campo NA/NaN
for col in fillna_mean:
  clean_train[col]._____(
      value=_____._____(),
      inplace=_____
  )

In [0]:
# Verificando as informações do nosso dataset
clean_train.info()

In [0]:
# Verificando a alteração dos valores nas estatísticas descritivas
clean_train.describe()

### Observações
Quais observações você pode fazer sobre os dados limpos em relação aos dados sujos?

## Normalização
Como o algoritmo **KNeighborsRegressor** funciona melhor se todos os dados estiverem na mesma escala. Portanto iremos normalizar os dados para o intervalo [0, 1]. Para isso iremos usar:
- [**MinMaxScaler**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html): transforma os dados para valores em um determinado intervalo. Por padrão é o intervalo que queremos: [0, 1].
  - [**fit_transform**](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler.fit_transform): aplica o _fit_ e, logo após, o _transform_ nos dados.
- [**pandas.DataFrame.columns**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html): retorna as _labels_ das colunas do _DataFrame_.
- [**pandas.DataFrame.values**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.values.html): retorna os valores do _DataFrame_.

**Exercícios**:
1. Aplique `MinMaxScaler().fit_transform()` nos dados do nosso _DataFrame_.
1. Crie um novo _DataFrame_ e passe as colunas em `columns`.

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
# Normalizando os dados
normalized_train = _____()._____(clean_train._____)
# Criamos um novo DataFrame com os dados normalizados
normalized_train = pd.DataFrame(normalized_train, columns=clean_train._____)
# Colocamos a coluna de preço para o seu valor real
normalized_train['price'] = clean_train['price']

In [0]:
# Verificamos as estatísticas descritivas do nosso novo DataFrame
normalized_train.describe()

## Outliers
Um _outlier_ é um valor que foge da normalidade dos dados que está inserido, podendo causar anomalias nos resultados obtidos nas análises e modelos treinados. Também são conhecidos como "pontos fora da curva".

Para realizarmos uma exploração inicial dos dados para identificar a existência de _outliers_, iremos usar:
- **[matplotlib](https://matplotlib.org/)**: é uma biblioteca de plotagem de gráficos.
- **[seaborn](https://seaborn.pydata.org/)**: é uma biblioteca de visualização de dados baseada no matplotlib. Ele fornece uma interface de alto nível para desenhar gráficos estatísticos atraentes e informativos.

In [0]:
# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

Iremos criar uma função chamada `print_eda` que irá nos mostrar alguns gráficos e valores que irão nos ajudar na etapa de exploração e identificação de _outliers_.

In [0]:
# Layout padrão
default_layout = (2,3)
# Figsize padrão
default_figsize = (10,4)

def plot_density(df):
  """Plota o gráfico de densidade.
  """
  df.plot(kind="density",
          subplots=True,
          sharex=False,
          layout=default_layout,
          figsize=default_figsize)
  plt.tight_layout()
  plt.show()

def plot_box(df):
  """Plota o boxplot.
  """
  df.plot(kind="box",
          subplots=True,
          sharex=False,
          layout=default_layout,
          figsize=default_figsize)
  plt.tight_layout()
  plt.show()
  
def print_eda(df):
  """Mostra gráficos e valores relacionados a etapa de
  Exploratory Data Analysis (EDA)."""
  hr = '--------------------------------------------------------------'
  print("Density")
  plot_density(df)

  print(hr)
  print("Box")
  plot_box(df)

  print(hr)
  print("Shape")
  print(df.shape)

  print(hr)
  print("Correlação")
  print(df.corr()['price'].sort_values())

  print(hr)
  print("Heatmap")
  sns.heatmap(df.corr())

In [0]:
# Mostrando os gráficos e valores dos dados que temos até agora
print_eda(normalized_train)

Para tratar esses valores, iremos usar duas técnicas e, posteriormente, escolher a que melhor comportou nos nossos dados. As técnicas serão:
- [Z-score](https://www.statisticshowto.datasciencecentral.com/probability-and-statistics/z-score/): nos permite comparar um valor específico com a população, levando-se em conta o valor típico e a dispersão.
- [IQR](https://pt-pt.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/interquartile-range-iqr/a/interquartile-range-review): também conhecida como amplitude interquartil, é uma medida da dispersão dos dados em torno da medida de centralidade.

In [0]:
from sklearn.preprocessing import StandardScaler

Estamos usando:
- [**pandas.DataFrame.copy**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html): cria uma cópia do _DataFrame_.
- [**pandas.DataFrame.quantile**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html): retorne valores no quantil fornecido sobre o eixo solicitado.
- [**pandas.DataFrame.all**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.all.html): retorne se todos os elementos são `True`, potencialmente sobre um eixo.

**Exercícios**:
- Para IQR:
  1. Copie o _DataFrame_.
  1. Pegue os valores que definem os eixos dos quartis 1 e 3.
- Para Z-Score:
  1. Crie o escalar com `StandardScaler` e aplique `fit_transform()` nos dados.

In [0]:
def clean_outliers(data, algorithm='iqr'):
  """Essa função limpa os outliers, podendo ser escolhido o algoritmo
  IQR ou Z-Score.
  
  Return: df
    O dataframe com os outliers tratados.
  """
  df = data._____()
  if algorithm == "iqr":
    Q1 = df._____(0.25)
    Q3 = df._____(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    up = Q3 + 1.5 * IQR
    df = df[((df > low).all(axis=1) & (df < up).all(axis=1))]
  elif algorithm == "z-score":
    df = pd.DataFrame(_____()._____(df),
                            columns=df.columns,
                            index=df.index)
    df = df[(df < 2.698).all(axis=1) & (df > -2.698).all(axis=1)]
  return df

In [0]:
train_iqr = clean_outliers(normalized_train,"iqr")
train_z_score = clean_outliers(normalized_train,"z-score")
train_z_score['price'] = normalized_train.loc[train_z_score.index.tolist()]['price']

In [0]:
print_eda(train_iqr)

In [0]:
print_eda(train_z_score)

### Comparação
Compare os dados pelo IQR e Z-Score. Converse com seus colegas para chegarem a uma conclusão.

# Modelo inicial
No treinamento inicial, iremos treinar 3 modelos:
1. Com os dados limpos e normalizados (**normalized_train**);
1. Com os dados após IQR (**train_iqr**);
1. Com os dados após Z-Score (**train_z_score**).

Iremos verificar o RMSE de cada um, para decidirmos qual _DataFrame_ usar. Após, na etapa de encontrar o melhor modelo, iremos aplicar vários modelos nos dados com o melhor RMSE. Por fim, iremos melhorar o nosso modelo final e aplicá-lo nas variáveis de teste.

Iremos usar:

- **[numpy.sqrt](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sqrt.html)**: calcula a raiz quadrada do valor passado.
- **[sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)**: calcula o erro médio quadrático.
- **[sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)**: divide matrizes em subconjuntos aleatórios de treino e teste.

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

## Treinamento inicial
A ordem dos modelos será igual a anteriormente citada.

In [0]:
# 10% para teste, 90% para treino
test_size = 0.10
# Semente usada pelo gerador de números aleatórios
seed = 20

Estamos usando:
- [**pandas.DataFrame.drop**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html): retirar linhas ou colunas usando rótulos.

**Exercícios**:
A seguir, você irá treinar 3 modelos básicos. Em todos eles, execute os seguintes passos:

- Em `train_test_split`:
  1. Passe, como primeiro argumento, todos dos dados de `normalized_train`, com exceção da coluna `price`.
  1. Passe, como segundo argumento, a coluna `price`.
  1. Para `test_size`, passe a variável `test_size`.
  1. Para `random_state`, passe `seed`.
- Em `KNeighborsRegressor`:
  1. Para `n_neighbors`, passe como argumento o valor `5`.
  1. Para `n_jobs`, passe como argumento o valor `-1`.
- Realize o `fit`.
- Realize o `predict`.
- Calcule o RMSE.

Ao final, compare o resultado dos modelos.

### Modelo com os dados limpos e normalizados
_DataFrame_ utilizado: `normalized_train`.

In [0]:
# Separamos o X e Y de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(normalized_train.drop(axis=1,labels=["price"]), 
                                                    normalized_train[_____],
                                                    test_size=_____, 
                                                    random_state=_____)
# Imprime as dimensões de X para treino e teste
print(X_train.shape, X_test.shape)

In [0]:
# Modelo básico inicial
knn = KNeighborsRegressor(_____=5, _____=-1)

# Treina o modelo básico
knn.fit(X_train, Y_train)

# Predizer os preços
predict = knn._____(X_test)

In [0]:
# RMSE
rmse = np.sqrt(_____(Y_test,predict))
print(rmse)

### Modelo com os dados após IQR
_DataFrame_ utilizado: `train_iqr`.

In [0]:
# Separamos o X e Y de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(train_iqr.drop(axis=1,labels=["price"]), 
                                                    train_iqr[_____],
                                                    _____=test_size, 
                                                    _____=seed)
# Imprime as dimensões de X para treino e teste
print(X_train.shape,X_test.shape)

In [0]:
# Modelo básico inicial
knn = KNeighborsRegressor(_____=5, n_jobs=_____)

# Treina o modelo básico
knn._____(X_train, Y_train)

# Predizer os preços
predict = knn.predict(X_test)

In [0]:
# RMSE
rmse = np._____(mean_squared_error(Y_test,predict))
print(rmse)

### Modelo com os dados após Z-Score
_DataFrame_ utilizado: `train_z_score`.

In [0]:
# Separamos o X e Y de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(_____, 
                                                    _____["price"],
                                                    _____=test_size, 
                                                    random_state=_____)
# Imprime as dimensões de X para treino e teste
print(X_train.shape,X_test.shape)

In [0]:
# Modelo básico inicial
knn = KNeighborsRegressor(n_neighbors=_____, n_jobs=_____)

# Treina o modelo básico
knn._____(X_train, Y_train)

# Predizer os preços
predict = knn._____(X_test)

In [0]:
# RMSE
rmse = np._____(_____(Y_test,predict))
print(rmse)

### Comparação
Compare os 3 modelos e verifique qual é o melhor entre eles.